# Vector Store 기반 검색기

Vector Store 기반 검색기는 vector store를 사용하여 문서를 검색하는 retriever입니다.

이는 vector store 클래스를 retriever 인터페이스에 맞게 래핑한 경량 래퍼(wrapper)입니다.

Vector store에 구현된 **유사도 검색(similarity search)** 이나 **MMR** 과 같은 검색 메서드를 사용하여 vector store 내의 텍스트를 쿼리합니다.


샘플 데이터를 `TextLoader` 로 로드합니다.


In [5]:
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 "./data/appendix-keywords.txt" 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

- `loader`를 사용하여 문서를 로드합니다.
- `CharacterTextSplitter`를 사용하여 로드된 문서를 분할하고, 분할된 텍스트를 `texts` 변수에 저장합니다.
- `OpenAIEmbeddings`를 사용하여 임베딩 객체를 생성합니다.
- `FAISS.from_documents` 메서드를 사용하여 `texts`와 `embeddings`로부터 FAISS 벡터 데이터베이스 `db`를 생성합니다.


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# 문서를 로드합니다.
documents = loader.load()
# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 1000이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
# 로드된 문서를 분할합니다.
texts = text_splitter.split_documents(documents)
# OpenAI 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()
# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다.
db = FAISS.from_documents(texts, embeddings)

`db.as_retriever()` 메서드를 호출하여 `retriever` 객체를 생성합니다.

- `as_retriever()` 메서드는 벡터 데이터베이스를 검색기(retriever)로 변환하여 반환합니다.
- 검색기(Retriever)는 질의(query)에 대해 유사도가 높은 문서를 검색하는 역할을 수행합니다.


In [8]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당합니다.
retriever = db.as_retriever()

`retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 질의에 관련된 문서를 검색합니다.

- `get_relevant_documents` 메서드는 질의를 입력으로 받습니다. 질의와 관련된 문서를 검색하고 반환합니다.


In [11]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
docs[0]

Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'})

## Max Marginal Relevance (MMR)

`MMR(Maximal Marginal Relevance)` 방식은 쿼리에 대한 관련 항목을 검색할 때 중복을 피하는 방법 중 하나입니다. 단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 검색된 항목들 사이에 **관련성과 다양성 사이의 균형을 보장**합니다. 이는 **자주 발생할 수 있는, 매우 유사한 항목들만이 검색되는 상황을 방지** 하는 데에 유용합니다.

예를 들어, 특정 주제에 대해 정보를 찾고 있다고 가정해봅시다. 가장 관련성 높은 문서만을 반환하는 시스템은 비슷비슷한 정보를 담은 문서들을 제공할 수 있습니다. 하지만, MMR 방식을 사용하면, 검색된 문서들이 해당 주제에 대해 서로 다른 관점이나 새로운 정보를 제공하도록 합니다. 이로써 사용자는 주제에 대해 보다 폭넓은 이해를 할 수 있게 됩니다.

MMR은 두 가지 주요 요소, 즉 쿼리에 대한 문서의 관련성과 **이미 선택된 문서들과의 차별성을 동시에 고려** 합니다.


- `db` 객체를 `as_retriever` 메서드를 사용하여 retriever로 변환합니다.
- `search_type` 매개변수를 `"mmr"` 로 설정하여 **MMR(Maximal Marginal Relevance)** 검색 알고리즘을 사용합니다.


In [12]:
# 데이터베이스를 검색기로 사용하며, MMR(Maximal Marginal Relevance) 검색 유형을 지정합니다.
retriever = db.as_retriever(search_type="mmr")

`retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 질의에 관련된 문서를 검색합니다.

- `get_relevant_documents` 메서드는 질의를 입력으로 받습니다. 질의와 관련된 문서를 검색하고 반환합니다.


In [13]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
docs[0]

Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'})

## 유사도 점수 임계값 검색기(Similarity Score Threshold Retrieval)

유사도 점수 임계값을 설정하고 해당 임계값 이상의 점수를 가진 문서만 반환하는 검색 방법을 설정할 수 있습니다.

임계값을 적절히 설정함으로써 **관련성이 낮은 문서를 필터링** 하고, 질의와 **가장 유사한 문서만 선별** 할 수 있습니다.


- `db.as_retriever()` 메서드를 사용하여 데이터베이스를 기반으로 하는 검색기(retriever)를 생성합니다.
- `search_type` 매개변수를 "similarity_score_threshold"로 설정하여 유사도 점수 임계값을 기준으로 검색을 수행합니다.
- `search_kwargs` 매개변수에 `{"score_threshold": 0.5}`를 전달하여 유사도 점수 임계값을 0.5로 설정합니다. 이는 검색 결과의 **유사도 점수가 0.5 이상인 문서만 반환됨** 을 의미합니다.


In [20]:
retriever = db.as_retriever(
    # 검색 유형을 "유사도 점수 임계값"으로 설정합니다.
    search_type="similarity_score_threshold",
    # 검색 인자로 점수 임계값을 0.5로 지정합니다.
    search_kwargs={"score_threshold": 0.75},
)

`retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 질의에 관련된 문서를 검색합니다.

- `get_relevant_documents` 메서드는 질의를 입력으로 받습니다. 질의와 관련된 문서를 검색하고 반환합니다.


In [21]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
# 전체 검색 결과를 출력합니다.
docs

[Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'}),
 Document(page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)', metadata={'source': './data/appendix-keywords.txt'}),
 Document(page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding', metadata={'source': './data/appendix-keywords.txt'})]

## Top K 명시

검색 시 사용할 `k`와 같은 검색 키워드 인자(kwargs)를 지정할 수 있습니다.

`k` 매개변수는 검색 결과에서 반환할 상위 결과의 개수를 나타냅니다. 적절한 `k` 값을 설정함으로써 검색 성능과 사용자 경험을 최적화할 수 있습니다.


- `db` 객체를 `retriever`로 변환합니다.
- `as_retriever` 메서드를 사용하여 검색 인자 `search_kwargs`를 전달합니다.
- `search_kwargs`에서 `k` 매개변수를 1로 설정하여 검색 결과로 반환할 문서의 수를 지정합니다.


In [22]:
# 데이터베이스를 검색하여 가장 관련성이 높은 1개의 결과를 반환하는 retriever를 생성합니다.
retriever = db.as_retriever(search_kwargs={"k": 1})

`retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 질의에 관련된 문서를 검색합니다.

- `len(docs)`를 통해 검색된 관련 문서의 개수를 확인합니다.


In [26]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
# 전체 검색 결과를 출력합니다.
print(docs)
# 문서의 개수를 출력합니다.
print(f"문서의 개수: {len(docs)}")

[Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'})]
문서의 개수: 1
